In [71]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF
import warnings
warnings.filterwarnings("ignore")
from fuzzywuzzy import process
import pickle

In [2]:
df=pd.read_csv('merged_df.csv',sep=',')

In [ ]:
df.head()

In [3]:
df.drop(columns=['movieId', 'genres', 'timestamp','Unnamed: 0' ], inplace=True)

In [16]:
df

,title,userId,rating
0,Toy Story (1995),1,4.0
1,Grumpier Old Men (1995),1,4.0
2,Heat (1995),1,4.0
3,Seven (a.k.a. Se7en) (1995),1,5.0
4,"Usual Suspects, The (1995)",1,5.0
...,...,...,...
100831,Split (2017),610,4.0
100832,John Wick: Chapter Two (2017),610,5.0
100833,Get Out (2017),610,5.0
100834,Logan (2017),610,5.0


In [17]:
unique_movies = df['title'].unique()
movie_list = df['title'].tolist()
movies_dict = {unique_movies[i] : i for i in range(len(unique_movies))}
print(movies_dict)
print(len(movies_dict))

{'Toy Story (1995)': 0, 'Grumpier Old Men (1995)': 1, 'Heat (1995)': 2, 'Seven (a.k.a. Se7en) (1995)': 3, 'Usual Suspects, The (1995)': 4, 'From Dusk Till Dawn (1996)': 5, 'Bottle Rocket (1996)': 6, 'Braveheart (1995)': 7, 'Rob Roy (1995)': 8, 'Canadian Bacon (1995)': 9, 'Desperado (1995)': 10, 'Billy Madison (1995)': 11, 'Clerks (1994)': 12, 'Dumb & Dumber (Dumb and Dumber) (1994)': 13, 'Ed Wood (1994)': 14, 'Star Wars: Episode IV - A New Hope (1977)': 15, 'Pulp Fiction (1994)': 16, 'Stargate (1994)': 17, 'Tommy Boy (1995)': 18, 'Clear and Present Danger (1994)': 19, 'Forrest Gump (1994)': 20, 'Jungle Book, The (1994)': 21, 'Mask, The (1994)': 22, 'Blown Away (1994)': 23, 'Dazed and Confused (1993)': 24, 'Fugitive, The (1993)': 25, 'Jurassic Park (1993)': 26, 'Mrs. Doubtfire (1993)': 27, "Schindler's List (1993)": 28, 'So I Married an Axe Murderer (1993)': 29, 'Three Musketeers, The (1993)': 30, 'Tombstone (1993)': 31, 'Dances with Wolves (1990)': 32, 'Batman (1989)': 33, 'Silence of 

In [4]:
ratings = df.pivot_table(index='userId',columns='title', values='rating')
ratings

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.5,3.5,NaN,NaN,NaN


In [5]:
# filling the missing data with mean value
def fill_nan(df):
    mean_values = df.mean()
    df.fillna(mean_values,inplace=True)
    return df

In [6]:
ratings = fill_nan(ratings)
ratings

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,4.0,3.5,5.0,4.0,1.5,3.176471,3.0,3.666667,3.285714,...,1.5,4.0625,3.666667,3.0,3.0,3.863636,2.770833,2.0,4.000000,1.0
2,4.0,4.0,3.5,5.0,4.0,1.5,3.176471,3.0,3.666667,3.285714,...,1.5,4.0625,3.666667,3.0,3.0,3.863636,2.770833,2.0,3.134615,1.0
3,4.0,4.0,3.5,5.0,4.0,1.5,3.176471,3.0,3.666667,3.285714,...,1.5,4.0625,3.666667,3.0,3.0,3.863636,2.770833,2.0,3.134615,1.0
4,4.0,4.0,3.5,5.0,4.0,1.5,3.176471,3.0,3.666667,3.285714,...,1.5,4.0625,3.666667,3.0,3.0,3.863636,2.770833,2.0,3.134615,1.0
5,4.0,4.0,3.5,5.0,4.0,1.5,3.176471,3.0,3.666667,3.285714,...,1.5,4.0625,3.666667,3.0,3.0,3.863636,2.770833,2.0,3.134615,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,4.0,4.0,3.5,5.0,4.0,1.5,3.176471,3.0,3.666667,3.285714,...,1.5,4.0625,3.666667,3.0,3.0,3.863636,2.770833,2.0,3.134615,1.0
607,4.0,4.0,3.5,5.0,4.0,1.5,3.176471,3.0,3.666667,3.285714,...,1.5,4.0625,3.666667,3.0,3.0,3.863636,2.770833,2.0,3.134615,1.0
608,4.0,4.0,3.5,5.0,4.0,1.5,3.176471,3.0,3.666667,3.285714,...,1.5,4.0625,3.666667,3.0,3.0,4.500000,3.500000,2.0,3.134615,1.0


In [63]:
# cleaning the titles
def clean_title(df):
    cols = df.columns
    new_columns = []
    for col in cols:
        new = col.lstrip().rstrip().replace ("'", "")
        new_columns.append(new)  
        
    df.columns = new_columns

In [64]:
clean_title(ratings)
ratings

,71 (2014),Hellboy: The Seeds of Creation (2004),Round Midnight (1986),Salems Lot (2004),Til There Was You (1997),Tis the Season for Love (2015),"burbs, The (1989)",night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,4.0,3.5,5.0,4.0,1.5,3.176471,3.0,3.666667,3.285714,...,1.5,4.0625,3.666667,3.0,3.0,3.863636,2.770833,2.0,4.000000,1.0
2,4.0,4.0,3.5,5.0,4.0,1.5,3.176471,3.0,3.666667,3.285714,...,1.5,4.0625,3.666667,3.0,3.0,3.863636,2.770833,2.0,3.134615,1.0
3,4.0,4.0,3.5,5.0,4.0,1.5,3.176471,3.0,3.666667,3.285714,...,1.5,4.0625,3.666667,3.0,3.0,3.863636,2.770833,2.0,3.134615,1.0
4,4.0,4.0,3.5,5.0,4.0,1.5,3.176471,3.0,3.666667,3.285714,...,1.5,4.0625,3.666667,3.0,3.0,3.863636,2.770833,2.0,3.134615,1.0
5,4.0,4.0,3.5,5.0,4.0,1.5,3.176471,3.0,3.666667,3.285714,...,1.5,4.0625,3.666667,3.0,3.0,3.863636,2.770833,2.0,3.134615,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,4.0,4.0,3.5,5.0,4.0,1.5,3.176471,3.0,3.666667,3.285714,...,1.5,4.0625,3.666667,3.0,3.0,3.863636,2.770833,2.0,3.134615,1.0
607,4.0,4.0,3.5,5.0,4.0,1.5,3.176471,3.0,3.666667,3.285714,...,1.5,4.0625,3.666667,3.0,3.0,3.863636,2.770833,2.0,3.134615,1.0
608,4.0,4.0,3.5,5.0,4.0,1.5,3.176471,3.0,3.666667,3.285714,...,1.5,4.0625,3.666667,3.0,3.0,4.500000,3.500000,2.0,3.134615,1.0


### recommend_nmf function

In [19]:
movie_title_dict={key:value for (value,key) in enumerate(movies_dict)}
movie_title_dict

{'Toy Story (1995)': 0,
 'Grumpier Old Men (1995)': 1,
 'Heat (1995)': 2,
 'Seven (a.k.a. Se7en) (1995)': 3,
 'Usual Suspects, The (1995)': 4,
 'From Dusk Till Dawn (1996)': 5,
 'Bottle Rocket (1996)': 6,
 'Braveheart (1995)': 7,
 'Rob Roy (1995)': 8,
 'Canadian Bacon (1995)': 9,
 'Desperado (1995)': 10,
 'Billy Madison (1995)': 11,
 'Clerks (1994)': 12,
 'Dumb & Dumber (Dumb and Dumber) (1994)': 13,
 'Ed Wood (1994)': 14,
 'Star Wars: Episode IV - A New Hope (1977)': 15,
 'Pulp Fiction (1994)': 16,
 'Stargate (1994)': 17,
 'Tommy Boy (1995)': 18,
 'Clear and Present Danger (1994)': 19,
 'Forrest Gump (1994)': 20,
 'Jungle Book, The (1994)': 21,
 'Mask, The (1994)': 22,
 'Blown Away (1994)': 23,
 'Dazed and Confused (1993)': 24,
 'Fugitive, The (1993)': 25,
 'Jurassic Park (1993)': 26,
 'Mrs. Doubtfire (1993)': 27,
 "Schindler's List (1993)": 28,
 'So I Married an Axe Murderer (1993)': 29,
 'Three Musketeers, The (1993)': 30,
 'Tombstone (1993)': 31,
 'Dances with Wolves (1990)': 32,
 

In [77]:
query = {'Emma (1996)': 1,
         'War of the Worlds (2005)': 2,
         'Confessions of a Dangerous Mind (2002)': 3,
         'Eros (2004)': 4,
         'Saturn 3 (1980)': 5}


def recommend_nmf(query, model, k=10):
    """Filters and recommends the top k movies 
    for any given input query based 
    on a trained NMF model.

    Parameters
    ----------
    query : dict
        A dictionary of movies already seen. 
        Takes the form {"movie_A": 3, "movie_B": 3} etc
    nmf_model : pickle
        pickle nmf_model read from disk
    k : int, optional
        no. of top movies to recommend, by default 10
    """
    # 1. candiate generation
    count = 0
    new_query = {}
    # construct a user vector
    for movie in ratings.columns:

        if movie not in query:
            # fuzzy = process.extractOne(movie, movie_title_dict.keys()) #matches the input
            new_query[movie] = np.nan
            # new_query.append(fuzzy[0])
        else:
            new_query[movie] = query[movie]

    new_query = pd.DataFrame(new_query, index=[0])
    new_query = np.array(new_query).reshape(1, -1)

    #print(new_query)

    user_dataframe = pd.DataFrame(new_query, index=[
                                  'Recommendation'], columns=ratings.columns).fillna(ratings.mean())
    #print(user_dataframe)

    # 2. scoring

    # calculate the score with the NMF nmf_model
    model = pickle.load(open("nmf_model.sav", "rb"))
    # calculate the score with the NMF model
    user_P = model.transform(user_dataframe)
    Q = model.components_
    user_R = np.dot(user_P, Q)

    user_dataframe = pd.DataFrame(
        user_R, index=['Recommendation'], columns=ratings.columns)

    # 3. ranking

    # set zero score to movies already seen by the user
    unrated_boolean = np.isnan(new_query)[0]
    unrated_df = user_dataframe.iloc[:, unrated_boolean]
    sorted_new_user_df = unrated_df.T.sort_values(
        by='Recommendation', ascending=False).head(k)

    return sorted_new_user_df.head(k)

In [78]:
recommend_nmf(query,'nmf_model.sav',k=10)

,Recommendation
More (1998),5.00262
Girls About Town (1931),5.00262
Crossing Delancey (1988),5.00262
Ooops! Noah is Gone... (2015),5.00262
Unfaithfully Yours (1948),5.00262
Reform School Girls (1986),5.00262
Sisters (Syostry) (2001),5.00262
Crippled Avengers (Can que) (Return of the 5 Deadly Venoms) (1981),5.00262
Little Murders (1971),5.00262
"Wings, Legs and Tails (1986)",5.00262
